In [1]:
# Importar librerías necesarias
import numpy as np

## Comprobación por Roché-Frobenius
    - Si Rank(A) <> Rank(A*) --> Incompatible
    - Si Rank(A*) == nº incognitas --> Compatible determinado
    - Si Rank(A*) < nº incognitas --> Compatible indeterminado

In [2]:
def check_rank(A, E):
    extended_rank = np.linalg.matrix_rank(E)
    if (np.linalg.matrix_rank(A) != extended_rank):
        raise ValueError("La matriz introducida es incompatible")
    if (extended_rank < A.shape[1]):
        raise ValueError("La matriz introducida es compatible indeterminado")

# **Práctica 6:** Resolución de sistemas de ecuaciones lineales con métodos iterativos (Jacobi y Gauss-Seidel)

El objetivo de esta práctica es programar los métodos de resolución iterativa de sistemas de ecuaciones lineales explicados en clase. 

Los programas deben recibir la matriz extendida del sistema $[A, b]$ (matriz de coeficientes y la última columna, la matriz de términos independientes).

**NOTAS:** 
* Los métodos a implementar en las secciones 1.1 y 1.2 deben lanzar una Excepción si se recibe como entrada un sistema incompatible o compatible indeterminado.
* Considerar un criterio de parada estudiado en la clase teórica. Considerar el error de $0.001$.
* Tener en cuenta que el método pueda no converger. Por tanto, establecer un
número máximo de iteraciones, por ejemplo $maxIter = 1000$
* Al final, el programa deberá dar un aviso en pantalla informando si ha encontrado la solución o si la parada fue por la no convergencia. 

# 1. Implementación de los métodos

## 1.1. Método de Jacobi

Implementa el método de Jacobi para resolver sistemas de ecuaciones lineales explicado en teoría.

In [3]:
def Jacobi(M, x_0):
    # Comprueba que sea compatible determinado
    size = M.shape[1] - 1
    try:
        check_rank(M[:,range(size)], M)
    except ValueError as exception:
        print(exception)
        return None
    
    # Declaramos los criterios de parada
    max_iters = 1000
    tol = 1e-9

    # Variables necesarias para la ejecucion
    op = 0
    coef_mat = M[:, range(size)]
    ind_matrix = M[:, size]
    pre_sol = x_0
    
    # Comienza el algoritmo
    for num_iter in range(max_iters):
        sol = np.zeros(M.shape[0])
        for i in range(size):
            op = 0
            for j in range(size):
                if i != j:
                    op += coef_mat[i, j] * pre_sol[j]
            sol[i] = (ind_matrix[i] - op) / coef_mat[i, i]
        if np.linalg.norm(pre_sol - sol) < tol:
            return sol
        pre_sol = sol    
    
    print("Jacobi termina por maximo de iteraciones")
    return None

## 1.2. Método de Gauss-Seidel

Implementa el método de Gauss-Seidel para resolver sistemas de ecuaciones lineales.

In [4]:
def GaussSeidel(M, x_0):
    # Comprueba que sea compatible determinado
    size = M.shape[1] - 1
    try:
        check_rank(M[:,range(size)], M)
    except ValueError as exception:
        print(exception)
        return None
    
    # Declaramos los criterios de parada
    max_iters = 1000
    tol = 1e-9

    # Variables necesarias para la ejecucion
    op1 = 0
    op2 = 0
    coef_mat = M[:, range(size)]
    ind_matrix = M[:, size]
    sol = x_0
 
    # Comienza el algoritmo
    for num_iter in range(max_iters):
        for i in range(size):
            op1 = np.dot(coef_mat[i, :i], sol[:i])
            op2 = np.dot(coef_mat[i, i + 1:], sol[i + 1:])
            sol[i] = (ind_matrix[i] - op1 - op2) / coef_mat[i, i]
        if np.linalg.norm(np.dot(coef_mat, sol) - ind_matrix) < tol:
            return sol

    print("Gauss-Seidel termina por maximo de iteraciones")
    return None

# 2. Aplicación de los métodos

En este apartado, utiliza uno de los dos métodos implementados en la sección anterior para obtener sus resultados.

NOTA: Intentar reescribir el sistema para garantizar la convergencia, si posible.


## 2.1. Ejercicio A

Resuelve los siguientes sistemas:

1.  $\begin{cases}
    2x_1 + 4x_3 = 0\\
    6x_1 - 3x_2 + 7x_3 = 1\\
    -4x_1 + 6x_2 + 2x_3 = 3
    \end{cases}$
    
3. $\begin{cases}
    6x_1 - 9x_2 + x_3 + 4x_5 = -5\\
    -2x_1 + 3x_2 - x_3 + 5x_4 + 9x_5 = -24\\
    x_2 - x_3 + 7x_4 + 2x_5 = -10\\
    5x_1 - 6x_2 + 8x_3 - x_4 = 5\\
    3x_1 + 7x_2 - 2x_3 + x_4 + 5x_5 = 2
    \end{cases}$

Devuelve un ndarray para cada sistema


In [5]:
sys1_coef = np.array([[2, 0, 4], 
                      [6., -3., 7], 
                      [-4, 6, 2]], dtype=float)
sys1_ind = np.array([[0], [1], [3]], dtype=float)
sys1_ext = np.hstack((sys1_coef, sys1_ind))
res_1_jb = Jacobi(sys1_ext, np.array([[0], [0], [0]], dtype=float))
res_1_gs = GaussSeidel(sys1_ext, np.array([[0], [0], [0]], dtype=float))

sys2_coef = np.array([[6, -9, 1, 0, 4], 
                      [-2, 3, -1, 5, 9], 
                      [0, 1, 1, 7, 2], 
                      [5, -6, 8, -1, 0], 
                      [3, 7, -2, 1, 5]], dtype=float)
sys2_ind = np.array([[-5], [-24], [-10], [5], [2]])
sys2_ext = np.hstack((sys2_coef, sys2_ind))
res_2_jb = Jacobi(sys2_ext, np.array([[0], [0], [0], [0], [0]], dtype=float))
res_2_gs = GaussSeidel(sys2_ext, np.array([[0], [0], [0], [0], [0]], dtype=float))

print(f'Solucion 1: Jacobi: {res_1_jb} Gauss-Seidel: {res_1_gs}')
print(f'Solucion 2: Jacobi: {res_2_jb} Gauss-Seidel: {res_2_gs}')

La matriz introducida es incompatible
La matriz introducida es incompatible
Jacobi termina por maximo de iteraciones
Gauss-Seidel termina por maximo de iteraciones
Solucion 1: Jacobi: None Gauss-Seidel: None
Solucion 2: Jacobi: None Gauss-Seidel: None


C:\Users\MSI\AppData\Local\Temp\ipykernel_15900\1075780012.py:27: RuntimeWarning: overflow encountered in double_scalars
  op += coef_mat[i, j] * pre_sol[j]
C:\Users\MSI\AppData\Local\Temp\ipykernel_15900\1075780012.py:27: RuntimeWarning: invalid value encountered in double_scalars
  op += coef_mat[i, j] * pre_sol[j]


## 2.2. Ejercicio B

Problema. Un bar va a renovar el mobiliario y te ha pedido asesoramiento. Desean instalar mesas de tres tamaños: 
* Pequeñas, de 5 asientos cada una
* Medianas premium, de 10 asientos cada una
* Grandes, de 20 asientos cada una

Desean que en total haya 50 mesas, con capacidad para 525 comensales. Se ha calculado que el coste de cada mesa pequeña (con sus asientos) es de 100 euros; la mediana (con asientos), más cara, 500 euros; y la grande (con asientos) 250 euros. Se tiene un presupuesto de 17550 euros.

Resolver: 

1. Plantear el sistema de ecuaciones a resolver
2. Utilizar el método de Jacobi o Gauss-Seidel para calcular cuántas mesas deberán encargarse. Prueba a inicializar la solución con $\mathbf{x}^{(0)} = (0,0,0)$ y luego con otra que satisfaga la primera ecuación como por ejemplo $\mathbf{x}^{(0)} = (10,30,10)$

1. Sistema de ecuaciones. Planteamiento:

In [6]:
## Sistema de ecuaciones
# x + y + z = 50
# 5x + 10y + 20z = 525
# 100x + 500y + 250z = 17550
## Con el fin de intentar garantizar convergencia
## reorganizamos el sistema para que al menos 2
## ecuaciones sean diagonalmente dominantes:
# x + y + z = 50
# 100x + 500y + 250z = 17550
# 5x + 10y + 20z = 525

eq_coef = np.array([[1, 1, 1],
                    [100, 500, 250], 
                    [5, 10, 20]], dtype=float)
eq_ind = np.array([[50], [17550], [525]], dtype=float)
eq_complete = np.hstack((eq_coef, eq_ind))

2. Resolución. Devuelve un ndarray para la solución del sistema.

In [7]:
x_0 = np.zeros(eq_complete.shape[0])
sol_jb_0 = Jacobi(eq_complete, x_0.copy())
sol_gs_0 = GaussSeidel(eq_complete, x_0.copy())


x_1 = np.array([[10], [30], [10]], dtype=float)
sol_jb_1 = Jacobi(eq_complete, x_1.copy())
sol_gs_1 = GaussSeidel(eq_complete, x_1.copy())

print(f'Solucion {x_0}: jacobi: {sol_jb_0} gauss-seidel: {sol_gs_0}')
print(f'Solucion {x_1}: jacobi: {sol_jb_1} gauss-seidel: {sol_gs_1}')

Gauss-Seidel termina por maximo de iteraciones
Solucion [0. 0. 0.]: jacobi: [13. 28.  9.] gauss-seidel: [13. 28.  9.]
Solucion [[10.]
 [30.]
 [10.]]: jacobi: [13. 28.  9.] gauss-seidel: None


## 2.3. Ejercicio C

Estudia la solución del sistema formado por 
$$ 4x_1 + x_2 = 1 $$
$$ 4x_2 + x_3 = 2 $$
$$ 4x_3 + x_4 = 3 $$
$$ ... $$
$$ 4x_{10} + x_{11} = 10 $$
$$ 4x_{11} = 11 $$


1. Representar las matrices $A$ y $b$ del sistema

In [8]:
coefs = np.zeros((11,11))
ind_terms = np.array([[i + 1] for i in range(11)])

for i in range(coefs.shape[0]):
    for j in range(coefs.shape[1]):
        if i == j:
            coefs[i, j] = 4
            if j + 1 < 11:
                coefs[i, j + 1] = 1

M = np.hstack((coefs, ind_terms))
print(M)

[[ 4.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  4.  1.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 0.  0.  4.  1.  0.  0.  0.  0.  0.  0.  0.  3.]
 [ 0.  0.  0.  4.  1.  0.  0.  0.  0.  0.  0.  4.]
 [ 0.  0.  0.  0.  4.  1.  0.  0.  0.  0.  0.  5.]
 [ 0.  0.  0.  0.  0.  4.  1.  0.  0.  0.  0.  6.]
 [ 0.  0.  0.  0.  0.  0.  4.  1.  0.  0.  0.  7.]
 [ 0.  0.  0.  0.  0.  0.  0.  4.  1.  0.  0.  8.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  4.  1.  0.  9.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  4.  1. 10.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  4. 11.]]


2. Obtener la solución de las variables $x_1, x_2, \ldots, x_{11}$. Devuelve un ndarray de 11 posiciones para la solución del sistema.

In [9]:
print(f'Por Jacobi: ({Jacobi(M,  np.array([[0] for i in range(11)], dtype=float))})')
print(f'Por Gauss: ({GaussSeidel(M,  np.array([[0] for i in range(11)], dtype=float))})')

Por Jacobi: ([0.16000056 0.35999775 0.560009   0.75996399 0.96014404 1.15942383
 1.36230469 1.55078125 1.796875   1.8125     2.75      ])
Gauss-Seidel termina por maximo de iteraciones
Por Gauss: (None)
